# Group 1  

In [1]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import scipy.ndimage.filters
import folium
import math
import geopy.distance

### 1. Read the data

In [2]:
df = pd.read_excel('LaptopSalesLL.xlsx')

In [3]:
df = df.dropna()

In [4]:
index = df[df['Customer Postcode'] == 'WC2 4HS'].index.tolist()
df = df.drop(index)

In [5]:
df = df.reset_index(drop=True)

In [6]:
len(df)

283418

In [7]:
df['Customer Latitude'] = df['Customer Latitude'].astype(float)
df['Customer Longitude'] = df['Customer Longitude'].astype(float)

In [8]:
df.dtypes

Date                      datetime64[ns]
Configuration                      int64
Customer Postcode                 object
Store Postcode                    object
Retail Price                     float64
Screen Size (Inches)               int64
Battery Life (Hours)               int64
RAM (GB)                           int64
Processor Speeds (GHz)           float64
Integrated Wireless?              object
HD Size (GB)                       int64
Bundled Applications?             object
customer X                         int64
customer Y                         int64
store X                          float64
store Y                          float64
Customer Latitude                float64
Customer Longitude               float64
Store Latitude                   float64
Store Longitude                  float64
dtype: object

In [9]:
df.head(5)

,Date,Configuration,Customer Postcode,Store Postcode,Retail Price,Screen Size (Inches),Battery Life (Hours),RAM (GB),Processor Speeds (GHz),Integrated Wireless?,HD Size (GB),Bundled Applications?,customer X,customer Y,store X,store Y,Customer Latitude,Customer Longitude,Store Latitude,Store Longitude
0,2008-01-01 00:01:19,163,EC4V 5BH,SE1 2BN,455.0,15,5,1,2.0,Yes,80,Yes,532041,180995,534057.0,179682.0,51.512579,-0.098731,51.500186,-0.070057
1,2008-01-01 00:02:52,320,SW4 0JL,SW12 9HD,545.0,15,6,1,2.0,No,300,No,529240,175537,528739.0,173080.0,51.464129,-0.140912,51.442082,-0.149046
2,2008-01-01 00:04:18,23,EC3V 1LR,E2 0RY,515.0,15,4,1,2.0,Yes,300,Yes,533095,181047,535652.0,182961.0,51.512699,-0.083435,51.529346,-0.045410
3,2008-01-01 00:04:40,169,SW1P 3AU,SE1 2BN,395.0,15,5,1,2.0,No,40,Yes,529902,179641,534057.0,179682.0,51.500787,-0.129897,51.500186,-0.070057
4,2008-01-01 00:06:04,365,EC4V 4EG,SW1V 4QQ,585.0,15,6,2,2.0,No,120,Yes,531684,180948,528924.0,178440.0,51.512120,-0.103737,51.490255,-0.144474


### 2. Dataframe store

In [10]:
store = df[['Store Postcode']+['Store Latitude']+['Store Longitude']]
store=store.drop_duplicates(subset=None,keep='first',inplace=False)
##store = store.dropna()
store = store.reset_index(drop=True)

In [11]:
store_freq = []
for i in store['Store Postcode'].tolist():
    store_freq.append(len(df[df['Store Postcode'] == i]))
    store['frequent'] = pd.DataFrame(store_freq)
store

,Store Postcode,Store Latitude,Store Longitude,frequent
0,SE1 2BN,51.500186,-0.070057,44488
1,SW12 9HD,51.442082,-0.149046,12673
2,E2 0RY,51.529346,-0.045410,30681
3,SW1V 4QQ,51.490255,-0.144474,43318
4,SE8 3JD,51.483307,-0.025775,16583
5,SW18 1NN,51.461802,-0.199785,14069
6,CR7 8LE,51.398191,-0.093631,6604
7,NW5 2QH,51.551091,-0.137239,31381
8,W4 3PH,51.485033,-0.279066,5500
9,SW1P 3AU,51.500787,-0.129897,51645


### 3. Dataframe Customer

In [12]:
df.head(1)

,Date,Configuration,Customer Postcode,Store Postcode,Retail Price,Screen Size (Inches),Battery Life (Hours),RAM (GB),Processor Speeds (GHz),Integrated Wireless?,HD Size (GB),Bundled Applications?,customer X,customer Y,store X,store Y,Customer Latitude,Customer Longitude,Store Latitude,Store Longitude
0,2008-01-01 00:01:19,163,EC4V 5BH,SE1 2BN,455.0,15,5,1,2.0,Yes,80,Yes,532041,180995,534057.0,179682.0,51.512579,-0.098731,51.500186,-0.070057


In [13]:
customer = df[['Customer Postcode']+['Customer Latitude']+['Customer Longitude']]
customer= customer.drop_duplicates(subset=None,keep='first',inplace=False)
customer = customer.reset_index(drop=True)

In [14]:
customer.head()

,Customer Postcode,Customer Latitude,Customer Longitude
0,EC4V 5BH,51.512579,-0.098731
1,SW4 0JL,51.464129,-0.140912
2,EC3V 1LR,51.512699,-0.083435
3,SW1P 3AU,51.500787,-0.129897
4,EC4V 4EG,51.512120,-0.103737


### 4. Create dictionary for store latitude and longitude

In [15]:
stla = {}
stlo = {}
for i in store['Store Postcode'].tolist():
    a = store[store['Store Postcode'] == i ]
    stla[i] = float(a['Store Latitude'])
    stlo[i] = float(a['Store Longitude'])

In [16]:
customer.head()

,Customer Postcode,Customer Latitude,Customer Longitude
0,EC4V 5BH,51.512579,-0.098731
1,SW4 0JL,51.464129,-0.140912
2,EC3V 1LR,51.512699,-0.083435
3,SW1P 3AU,51.500787,-0.129897
4,EC4V 4EG,51.512120,-0.103737


### 5. Create dictionary for customers

In [17]:
dla = {}
dlo = {}

for i in store['Store Postcode'].tolist():
    one_store = df[df['Store Postcode'] == i]
    cu= one_store['Customer Postcode'].unique().tolist()
    
    temp = pd.DataFrame(columns=customer.columns.tolist())
    
    for index, row in customer.iterrows():
        if row['Customer Postcode'] in cu:
            temp = temp.append(row, ignore_index=True)
    
    dla[i] = temp['Customer Latitude'].tolist()
    dlo[i] = temp['Customer Longitude'].tolist()

### 6. Draw the map and make it interactive

In [18]:
for index, row in store.iterrows():
   
    # Start a map callled Loncon
    London = folium.Map(location=[51.5, -0.07],
                        zoom_start=13,
                        tiles="stamenterrain")
   
    #define color
    color="#FFA500"
    
    popup_text = "Store Postcode: {}<br> Store Latitude: {}<br> Store Longitude: {}<br> Frequent: {}"
    popup_text = popup_text.format(row["Store Postcode"],
                          round(row["Store Latitude"],2),
                          round(row["Store Longitude"],2),
                          row["frequent"])
    
    radius =math.sqrt(row["frequent"])/5
    # drew the circle of stores on the map
    folium.CircleMarker(location=(row["Store Latitude"],
                                      row["Store Longitude"]),
                                      radius=radius,
                                      color = color,
                                      popup=popup_text,
                                      fill = True).add_to(London)
    
    clat = dla[row['Store Postcode']]
    clon = dlo[row['Store Postcode']]
    
    # drew the circles of customers on the map
    for i in range(len(clat)):
        color="#1E90FF"
        popup_text = "Customer Postcode: {}<br> Customer Latitude: {}<br> Customer Longitude: {}"
        popup_text = popup_text.format(i,
                             round(clat[i],2),
                             round(clon[i],2))



        folium.PolyLine(locations=[[clat[i], clon[i]],[stla[row['Store Postcode']],stlo[row['Store Postcode']]]], color='forestgreen').add_to(London)
        
        folium.CircleMarker(location=(clat[i],
                                          clon[i]),
                                          color = color,
                                          radius= 3,
                                          popup=popup_text,
                                          fill = True).add_to(London)
        
        
    # Select other stores
    store1 = store
    store1 = store.drop(index)
    for index1, row1 in store1.iterrows():
        color="#FFA500"
        # make it interactive
        popup_text = "<a href=\"" + 'store' + row1['Store Postcode'] + '.html'  + "\">" + 'CLICK ME !' + "</a>"
        
        tooltip = "Store Postcode: {}<br> Store Latitude: {}<br> Store Longitude: {}<br> Frequent: {}"
        tooltip = tooltip.format(row["Store Postcode"],
                          round(row1["Store Latitude"],2),
                          round(row1["Store Longitude"],2),
                          row1["frequent"])
    
        radius =math.sqrt(row1["frequent"])/5
    
        folium.CircleMarker(location=(row1["Store Latitude"],
                                      row1["Store Longitude"]),
                                      radius=radius,
                                      color = color,
                                      popup=popup_text,
                                      tooltip = tooltip,
                                      fill = True).add_to(London)

    
        
    mymap = 'store' + row['Store Postcode'] + '.html' 
        
        
    London.save(mymap)
        

__<font color = green >Then, the maps has been saved as html format, you can start with any map to explore, click on an store to see it's customers! </font>__